In [0]:
#PREPROCESSING

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score, f1_score


carac = pd.read_csv("carac.csv",sep=';')
lieux = pd.read_csv("lieux.csv",sep=';')
veh = pd.read_csv("veh.csv",sep=';')
vict = pd.read_csv("vict.csv",sep=';')

victime = vict.merge(veh,on=['Num_Acc','num_veh'])
accident = carac.merge(lieux,on = 'Num_Acc')
victime = victime.merge(accident,on='Num_Acc')

hrmn=pd.cut(victime['hrmn'],24,labels=[str(i) for i in range(0,24)])


/root/.ipykernel/2574/command-3439463778833989-3833454651:19: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv("lieux.csv",sep=';')


In [0]:
#Diviser les données en train et test

#On supprime la colonne à prédire et l'ajoute dans Y
X = victime.drop('grav', axis=1)
y = victime['grav']

#Diviser les données en ensemble d'apprentissage et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Afficher les dimensions des ensembles
print("Dim de X_train :", X_train.shape)
print("Dim de X_test :", X_test.shape)
print("Dim de y_train :", y_train.shape)
print("Dim de y_test :", y_test.shape)

Dim de X_train : (102360, 50)
Dim de X_test : (25591, 50)
Dim de y_train : (102360,)
Dim de y_test : (25591,)


In [0]:
print(y_train.info)

<bound method Series.info of 30662     1
95953     4
14421     1
14755     3
33603     4
         ..
119879    1
103694    1
860       3
15795     1
121958    4
Name: grav, Length: 102360, dtype: int64>


In [0]:
from sklearn.neighbors import KNeighborsClassifier


neighbours_class = KNeighborsClassifier()

# la liste des paramètres à rechercher
param_grid = {'n_neighbors': [1, 3, 5, 7, 9]}

# on utilise le grid pour trouver les meilleurs paramètres
grid_search = GridSearchCV(neighbours_class, param_grid)
grid_search.fit(X_train, y_train)

# on les affiche pour ensuite les récupérer
print("liste meilleurs paramètres :", grid_search.best_params_)

# on fait le classifieur avec les meilleurs paramètres
best_neighbours_class = grid_search.best_estimator_

# on applique le classifieur (fait avec les meilleurs paramètres)
y_pred = best_neighbours_class.predict(X_test)

# et on affiche le score final
accuracy = accuracy_score(y_test, y_pred)
print("score sur l'ensemble de test :", accuracy)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-3439463778833996>, line 11
      9 # on utilise le grid pour trouver les meilleurs paramètres
     10 grid_search = GridSearchCV(neighbours_class, param_grid)
---> 11 grid_search.fit(X_train, y_train)
     13 # on les affiche pour ensuite les récupérer
     14 print("liste meilleurs paramètres :", grid_search.best_params_)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py:29, in _create_patch_function.<locals>.patch_function(self, *args, **kwargs)
     27 try:
     28     original_succeeded = False
---> 29     original_result = original(self, *args, **kwargs)
     30     original_succeeded = True
     31     return original_result

File /databricks/python/lib/python3.10/site-packages/sklearn/model_selection/_search.py:875, in BaseSearchCV.fit(self, X, y, groups, **fit_params)
    869     resul

In [0]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()

# on définit les paramètres à rechercher
param_grid = {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_leaf': [1, 2, 5, 10, 20]}

# on utilise le grid pour trouver les meilleurs paramètres
grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# on les affiche pour ensuite les récupérer
print("liste meilleurs paramètres :", grid_search.best_params_)

# on fait le classifieur avec les meilleurs paramètres
best_dt_model = grid_search.best_estimator_

# on applique le classifieur (fait avec les meilleurs paramètres)
y_pred = best_dt_model.predict(X_test)

# et on affiche le score final
accuracy = accuracy_score(y_test, y_pred)
print("score sur l'ensemble de test :", accuracy)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-3439463778833998>, line 10
      8 # on utilise le grid pour trouver les meilleurs paramètres
      9 grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='accuracy')
---> 10 grid_search.fit(X_train, y_train)
     12 # on les affiche pour ensuite les récupérer
     13 print("liste meilleurs paramètres :", grid_search.best_params_)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py:29, in _create_patch_function.<locals>.patch_function(self, *args, **kwargs)
     27 try:
     28     original_succeeded = False
---> 29     original_result = original(self, *args, **kwargs)
     30     original_succeeded = True
     31     return original_result

File /databricks/python/lib/python3.10/site-packages/sklearn/model_selection/_search.py:875, in BaseSearchCV.fit(self, X, y, groups, **fit_params)

In [0]:
rf_model = RandomForestClassifier()

# on définit les paramètres à rechercher
param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30, 40, 50],'min_samples_leaf': [1, 2, 5, 10, 20]}

# on utilise le grid pour trouver les meilleurs paramètres
grid_search = GridSearchCV(rf_model, param_grid)
grid_search.fit(X_train, y_train)

# on les affiche pour ensuite les récupérer
print("liste meilleurs paramètres :", grid_search.best_params_)

# on fait le classifieur avec les meilleurs paramètres
best_rf_model = grid_search.best_estimator_

# on applique le classifieur (fait avec les meilleurs paramètres)
y_pred = best_rf_model.predict(X_test)

# et on affiche le score final
accuracy = accuracy_score(y_test, y_pred)
print("score sur l'ensemble de test :", accuracy)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-3439463778833999>, line 8
      6 # on utilise le grid pour trouver les meilleurs paramètres
      7 grid_search = GridSearchCV(rf_model, param_grid)
----> 8 grid_search.fit(X_train, y_train)
     10 # on les affiche pour ensuite les récupérer
     11 print("liste meilleurs paramètres :", grid_search.best_params_)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py:29, in _create_patch_function.<locals>.patch_function(self, *args, **kwargs)
     27 try:
     28     original_succeeded = False
---> 29     original_result = original(self, *args, **kwargs)
     30     original_succeeded = True
     31     return original_result

File /databricks/python/lib/python3.10/site-packages/sklearn/model_selection/_search.py:875, in BaseSearchCV.fit(self, X, y, groups, **fit_params)
    869     results = self

In [0]:
# Modèle KNeighborsClassifier
y_pred_knn = best_neighbours_class.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

print("liste deds métriques pour le modèle knn :")
print("accuracy :", accuracy_knn)
print("f1-score :", f1_knn)

# Modèle DecisionTreeClassifier
y_pred_dt = best_dt_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

print("liste deds métriques pour le modèle knn :")
print("accuracy :", accuracy_dt)
print("f1-score :", f1_dt)

# Modèle RandomForestClassifier
y_pred_rf = best_rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print("liste deds métriques pour le modèle knn :")
print("accuracy :", accuracy_rf)
print("f1-score :", f1_rf)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3439463778834000>, line 2
      1 # Modèle KNeighborsClassifier
----> 2 y_pred_knn = best_neighbours_class.predict(X_test)
      3 accuracy_knn = accuracy_score(y_test, y_pred_knn)
      4 f1_knn = f1_score(y_test, y_pred_knn)

NameError: name 'best_neighbours_class' is not defined